In [ ]:
import pandas as pd
import requests

In [44]:
def get_bangkok_districts():
    bankok_osm_id = 92277
    bankok_area_id = 3600000000 + bankok_osm_id
    
    query = f"""
    [out:json][timeout:120];
    area({bankok_area_id})->.bangkok;
    relation["admin_level"="6"](area.bangkok);
    out tags;
    """

    url = "https://overpass-api.de/api/interpreter"
    response = requests.post(url, data={'data': query})
    data = response.json()

    districts = {}

    for elem in data.get("elements", []):
        name_th = elem["tags"].get("name:th")
        name_en = elem["tags"].get("name:en")
        name_local = elem["tags"].get("name")

        key = name_en or name_th or name_local

        districts[key] = {
            "osm_id": elem["id"],
            # "admin_level": elem["tags"].get("admin_level"),
            # "bbox": elem.get("bounds"),
            "name_th": name_th,
            "name_en": name_en,
            # "name_local": name_local
        }

    return districts

In [45]:
districts = get_bangkok_districts()

In [46]:
for k, v in districts.items():
    print(k, "=>", v)


Phra Nakhon District => {'osm_id': 92053, 'name_th': 'เขตพระนคร', 'name_en': 'Phra Nakhon District'}
Samphanthawong District => {'osm_id': 92056, 'name_th': 'เขตสัมพันธวงศ์', 'name_en': 'Samphanthawong District'}
Pom Prap Sattru Phai District => {'osm_id': 92058, 'name_th': 'เขตป้อมปราบศัตรูพ่าย', 'name_en': 'Pom Prap Sattru Phai District'}
Bang Rak District => {'osm_id': 92061, 'name_th': 'เขตบางรัก', 'name_en': 'Bang Rak District'}
Pathum Wan District => {'osm_id': 92063, 'name_th': 'เขตปทุมวัน', 'name_en': 'Pathum Wan District'}
Ratchathewi District => {'osm_id': 92064, 'name_th': 'เขตราชเทวี', 'name_en': 'Ratchathewi District'}
Phaya Thai District => {'osm_id': 92068, 'name_th': 'เขตพญาไท', 'name_en': 'Phaya Thai District'}
Dusit District => {'osm_id': 92069, 'name_th': 'เขตดุสิต', 'name_en': 'Dusit District'}
Bang Sue District => {'osm_id': 1639285, 'name_th': 'เขตบางซื่อ', 'name_en': 'Bang Sue District'}
Chatuchak District => {'osm_id': 1639286, 'name_th': 'เขตจตุจักร', 'name_en'

In [47]:
print("Total districts found:", len(districts))

Total districts found: 50


In [48]:
def get_hospital_districts(osm_id):
    """Return a list of hospitals for a district by OSM ID"""
    area_id = 3600000000 + osm_id
    query = f"""
    [out:json][timeout:120];
    area({area_id})->.searchArea;
    (
        node["amenity"="hospital"](area.searchArea);
        way["amenity"="hospital"](area.searchArea);
        relation["amenity"="hospital"](area.searchArea);
    );
    out center;
    """

    url = "https://overpass-api.de/api/interpreter"
    response = requests.post(url, data={"data": query})
    data = response.json()

    for k,v in districts.items():
        if v["osm_id"] == osm_id:
            district_name = k
            break

    hospital_num = 0

    hospitals = []
    
    for elem in data.get("elements", []):
        tags = elem.get("tags", {})
        name = tags.get("name:en") or tags.get("name")
        if not name:
            name = f"{district_name}school{hospital_num}"
            hospital_num += 1
        
        # Get coordinates
        if elem["type"] == "node":
            lat = elem.get("lat")
            lon = elem.get("lon")
        else:
            center = elem.get("center")
            lat = center.get("lat") if center else None
            lon = center.get("lon") if center else None

        if name and lat and lon:  # only include hospitals with coordinates
            hospitals.append({"name": name, "latitude": lat, "longitude": lon})

    return hospitals

In [49]:
hospital = get_hospital_districts(92053)

for i, v in enumerate(hospital, start=1):
    print(f"{i} => {v}")


1 => {'name': 'The Red Cross Health Station 2', 'latitude': 13.7519317, 'longitude': 100.4972541}


In [50]:
hospitals_by_district = {}

for district_name, info in districts.items():
    hospitals_by_district[district_name] = get_hospital_districts(info["osm_id"])

In [51]:
number_of_hospitals = 0
for k in hospitals_by_district:
    number_of_hospitals += len(k)
    
print("Total number of hospitals in Bangkok:", number_of_hospitals)

Total number of hospitals in Bangkok: 953


In [52]:
def get_school_districts(district_osm_id):
    district_area_id = 3600000000 + district_osm_id
    
    query_school = f"""
    [out:json][timeout:120];
    area({district_area_id})->.searchArea;
    (
        node["amenity"="school"](area.searchArea);
        way["amenity"="school"](area.searchArea);
        relation["amenity"="school"](area.searchArea);
    );
    out center;
    """
    
    query_university = f"""
    [out:json][timeout:120];
    area({district_area_id})->.searchArea;
    (
        node["amenity"="university"](area.searchArea);
        way["amenity"="university"](area.searchArea);
        relation["amenity"="university"](area.searchArea);
    );
    out center;
    """

    url = "https://overpass-api.de/api/interpreter"
    data_school = requests.post(url, data={'data': query_school}).json()
    data_university = requests.post(url, data={'data': query_university}).json()
    
    for k,v in districts.items():
        if v["osm_id"] == district_osm_id:
            district_name = k
            break

    school_num = 1
    

    schools_list = []

    for elem in data_school.get("elements", []) + data_university.get("elements", []):
        tags = elem.get("tags", {})
        name = tags.get("name:th") or tags.get("name_local") or tags.get("name") or tags.get("name:en")
        
        if not name:
            name = f"{district_name} school {school_num}"
            school_num += 1
        
        if elem["type"] == "node":
            lat = elem.get("lat")
            lon = elem.get("lon")
        else:
            center = elem.get("center", {})
            lat = center.get("lat")
            lon = center.get("lon")
        
        if lat and lon:
            schools_list.append({
                "osm_id": elem["id"],
                "name": name,
                "latitude": lat,
                "longitude": lon
            })

    return schools_list

In [53]:
school = get_school_districts( 92053)

for i, v in enumerate(school, start=1):
    print(f"{i} => {v}")

1 => {'osm_id': 1189577738, 'name': 'โรงเรียนวัดราชนัดดาราม', 'latitude': 13.7554133, 'longitude': 100.503825}
2 => {'osm_id': 3894809422, 'name': 'Baanchao Yoga', 'latitude': 13.7632303, 'longitude': 100.4947978}
3 => {'osm_id': 6276434287, 'name': 'Phra Nakhon District school 1', 'latitude': 13.7510215, 'longitude': 100.5020515}
4 => {'osm_id': 6287460721, 'name': 'โรงเรียนวัดใหม่อมตรส', 'latitude': 13.764855, 'longitude': 100.5014988}
5 => {'osm_id': 6391901412, 'name': 'Phra Nakhon District school 2', 'latitude': 13.7542865, 'longitude': 100.4987897}
6 => {'osm_id': 88336374, 'name': 'โรงเรียนพระปริยัติธรรม หอติสสเทวาภิธาน', 'latitude': 13.7490585, 'longitude': 100.5002613}
7 => {'osm_id': 203053776, 'name': 'โรงเรียนมัธยมวัดมกุฎกษัตริย์', 'latitude': 13.7642131, 'longitude': 100.5079095}
8 => {'osm_id': 234351081, 'name': 'Kumon', 'latitude': 13.7626913, 'longitude': 100.4968998}
9 => {'osm_id': 342454536, 'name': 'โรงเรียนวัดสังเวช', 'latitude': 13.7639615, 'longitude': 100.49715

In [54]:
schools_by_district = {}

for district_name, info in districts.items():
    schools_by_district[district_name] = get_school_districts(info["osm_id"])

In [55]:
row = 0

for k in schools_by_district:
    row += len(k)
    
print("Total number of schools in Bangkok:", row)

Total number of schools in Bangkok: 953


In [56]:
import pandas as pd

rows = []

for district, schools in schools_by_district.items():
    for school in schools:
        rows.append({
            "district": district,
            "name": school.get("name"),
            # "name_th": school.get("name_th"),
            # "name_local": school.get("name_local"),
            "latitude": school.get("latitude"),
            "longitude": school.get("longitude"),
            # "osm_id": school.get("osm_id")
        })

df = pd.DataFrame(rows)

df.to_csv("bangkok_schools.csv", index=False, encoding="utf-8-sig")

print("Saved CSV with", len(df), "rows")


Saved CSV with 736 rows


In [57]:
rows = []

for district, hospitals in hospitals_by_district.items():
    for hospital in hospitals:
        rows.append({
            "district": district,
            "name": hospital.get("name"),
            # "name_th": hospital.get("name_th"),
            # "name_local": hospital.get("name_local"),
            "latitude": hospital.get("latitude"),
            "longitude": hospital.get("longitude"),
            # "osm_id": hospital.get("osm_id")
        })

df = pd.DataFrame(rows)

df.to_csv("bangkok_hospital.csv", index=False, encoding="utf-8-sig")